In [ ]:
import hashlib
import math
import random
import numpy as np
import time

# Code pour collision entre MD5 et SHA256

In [ ]:
def trail(fhash,  k , l):
    """
    retourne un triplet (x0, xd, d) 
    fhash: une fonction d'hachage quelconque (md5, sha256)
    k: longueur de la chaine générer
    l: condition d'arret pour le point distingué : les l derniers bits doit etre nul
    """
    x0 = random.getrandbits(k)
    tmp = x0 # c'est un int
    d = 0
    max_it = 20/ (1/(2**l))  # d'apres l'article
    # 2**(l+2)
    mask_k = 2**k - 1
    mask_l = 2**l - 1
    while True:
        if d == max_it:
            #print("Risque de cycle")
            return None
        
        if tmp & mask_l == 0:  # condition d'arret
            xd = tmp
            break
        d += 1
        x = tmp.to_bytes(16, byteorder='big')  
        y = fhash(x).digest()
        tmp = int.from_bytes(y, byteorder='big') & mask_k
        #print(d, tmp)
    
    return (x0, xd, d)

In [ ]:
def f_cut_k(fhash, val, k):
    """
    Retourne la valeur retournée par fhash(val) tronquée à k bits
    fhash
    val : int
    k
    """
    x = val.to_bytes(16, byteorder='big')  
    y = fhash(x).digest()
    mask_k = 2**k - 1
    return int.from_bytes(y, byteorder='big') & mask_k

In [ ]:
def remonter (fhash ,A , B, k, b):
    """
    returne (x,y) tq x != y et fhash(x) == fhash(y)
    A, B : triplet (x0, xd, d)
    b : 0 ou 1
    """       
    if A[2] >= B[2]: 

        x = A[0]
        for _ in range(A[2]-B[2]):
            x = f_cut_k(fhash(b), x, k)
        y = B[0]

        if x == y : 
            #print('pb : x==y et fhash(x)==fhash(y)')
            return None

        while True:
            if x == y :
                break

            tmp1 = x            
            tmp2 = y             # anciennes valeurs
            x = f_cut_k(fhash(b), tmp1, k)  
            y = f_cut_k(fhash(1-b), tmp2, k)  

        return ( (tmp1, fhash(b).__name__ ) , (tmp2, fhash(1-b).__name__ ))
        #return (tmp1, tmp2)
        
    else:    # A[2] < B[2] mais on fait la meme chose
        
        y = B[0]
        for _ in range(B[2]-A[2]):
            y = f_cut_k(fhash(1-b), y, k)
        x = A[0]

        if x == y : 
            #print('pb : x==y et fhash(x)==fhash(y)')
            return None

        while True:
            if x == y :
                break

            tmp1 = x            
            tmp2 = y             # anciennes valeurs
            x = f_cut_k(fhash(b), tmp1, k) 
            y = f_cut_k(fhash(1-b), tmp2, k)  

        return ( (tmp1, fhash(b).__name__ ) , (tmp2, fhash(1-b).__name__ ))
        #return (tmp1, tmp2)

In [ ]:
def F(b):
    """
    Choisir uniformement une fonction de hash
    b : 0 ou 1
    """
    if b == 0:
        return hashlib.md5
    if b == 1:
        return hashlib.sha256

In [ ]:
def collision_detection2(fhash,  k, l):
    """
    detecte une seule collision
    fhash : prend un int en parametre et returne un pointer de fonction
            de hash
    retourne le couple de triplet ( (x0,xd,d), (x0',xd,d') )
    """
    dico = {}
    while True: 
        b = random.randint(0,1) 
        res = trail(fhash(b), k, l)
        if res == None:
            continue
            
        x0, xd, d = res
        
        if (xd,1-b) in dico:  
            #print("Collision found")
            A = (x0,xd,d)                                  # b
            B = (dico[(xd,1-b)][0], xd,dico[(xd,1-b)][1])  # 1-b
            return remonter(fhash, A , B, k, b )
        dico[(xd,b)] = (x0, d)


In [ ]:
def collision_detection_multiple(fhash, k, l, nb_col):
    liste = []
    i = 0
    while i<nb_col :
        tmp = collision_detection2(fhash,  k, l)
        if tmp == None:
            continue
        if tmp in liste or (tmp[1],tmp[0]) in liste: # si collision deja trouvé 
            continue 
        liste.append(tmp)
        i += 1
        #print((liste))
    return liste

In [ ]:
# collision_detection_multiple(F, 20, 10, (2**10)/2 )

In [ ]:
k = 20
l = 10

collision_detection2(F, k, l)

In [ ]:
print(f_cut_k(hashlib.md5, 222001, 20),'\n')
print(f_cut_k(hashlib.sha256, 913729, 20))